In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import cv2

import torch

from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch.nn as nn
from tensorflow.keras.models import load_model
import cv2
import numpy as np

In [2]:
import pretrainedmodels

def get_model(pretrained):
    if pretrained:
        model = pretrainedmodels.__dict__["resnet18"](
                pretrained="imagenet"
        )
        
    else:
        model = pretrainedmodels.__dict__["resnet18"](
                pretrined = None
        )
        
    model.last_linear = nn.Sequential(
                nn.BatchNorm1d(512),
            nn.Dropout(p=0.25),
            nn.Linear(in_features = 512, out_features = 100 ),
            nn.ReLU(),
                nn.BatchNorm1d(100 ,eps=1e-05, momentum=0.1),
            nn.Dropout(p=0.5),
            nn.Linear(in_features = 100, out_features = 1 ),
       nn.Sigmoid()
            )
    return model

In [3]:
model = get_model(pretrained=True)

In [4]:
model.load_state_dict(torch.load('model_new', map_location=torch.device('cpu')))

<All keys matched successfully>

In [5]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [6]:
import torch

from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

class dataset:
    def __init__(self, image, resize = None, augmentation = None):
        
        self.image = image
        
        self.resize = resize
        self.augmentation = augmentation
        
 
    def __getitem__(self):
        
        image = self.image
        
        image = cv2.resize(image,(100,100))
        
            
        image = np.array(image)
        
        if self.augmentation is not None:
            augmented = self.augmentation(image=image)
            image = augmented["image"]
            
            
        image = np.transpose(image, (2,0,1)).astype(np.float32)
            
        return torch.tensor(image, dtype= torch.float)
        

In [7]:

import albumentations
from sklearn import metrics
from sklearn.model_selection import train_test_split


mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

aug = albumentations.Compose(
[
    albumentations.Normalize(
    mean, std, max_pixel_value=255.0, always_apply= True
    )
]
)

In [ ]:
while(True):
    

    
    
    
    
    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)
    
   
    
    
    
    
    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        face_img = cv2.cvtColor(face_img, cv2.COLOR_GRAY2BGR)


        reshaped = dataset(image= face_img,  resize = (100,100), augmentation=aug)
        image = reshaped.__getitem__()
        

        image = np.reshape(image,(1,3,100,100))

        model.eval()
        
        result = model(image)
        result = result.detach().numpy()
        
        if result>0.9:
            label=0

        elif result<0.9:
            label=1
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()